In [1]:
import tensorflow_datasets as tfds

/home/polivares/anaconda3/envs/DataScience/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
dataset, info = tfds.load("tf_flowers", as_supervised=True, with_info=True)

In [3]:
test_set, valid_set, train_set = tfds.load("tf_flowers",split=["train[0%:10%]", "train[10%:25%]", "train[25%:]"], as_supervised=True )

In [4]:
import tensorflow as tf

In [6]:
# Previo a la utilización de nuestro dataset en la red neuronal, la red que utilizaremos provee de
# funcionalidades de pre-procesamiento ya incluídas. Por lo tanto, utilizaremos estas funcionalidades
# y las dejaremos dentro de una función
def preprocess(image, label):
    # Cambiaremos las dimensiones de la imagen de entrada
    resized_image = tf.image.resize(image, [224, 224]) # Guardamos la imagen con nuevas dimensiones 224x224
    # Luego pasamos la imagen modificada en tamaño al preprocesamiento de nuestra red. La red
    # que utilizaremos de ejemplo tiene por nombre Xception
    final_image = tf.keras.applications.xception.preprocess_input(resized_image)
    # Finalmente se retorna una imagen pre procesada (según lo indique el preprocess de xception)
    # y su etiqueta
    return final_image, label

In [7]:
# En este apartado hacemos los batch de datos directamente desde el dataset cargado por tensorflow
batch_size = 32

# Mezclamos el dataset 
train_set = train_set.shuffle(1000)
# Tanto para training, test y validación, aplicamos la función de preprocesamiento (preprocess)
# y luego generamos los batch de datos
train_set = train_set.map(preprocess).batch(batch_size).prefetch(1)
test_set = test_set.map(preprocess).batch(batch_size).prefetch(1)
valid_set = valid_set.map(preprocess).batch(batch_size).prefetch(1)

In [8]:
# Acá empezamos con transfer learning. Lo que haremos será cargar una arquitectura de red neuronal desde
# tensorflow ya entrenada. Eso quiere decir que, no solo estamos cargando la arquitectura, con sus neuronas
# y conexiones, sino que también estamos cargando los PESOS DE ENTRENAMIENTO.

# Weights indica si utilizaremos pesos pre entrenados con el dataset imagenet o no
# include_top es el parámetro que indica explícitamente si quieres o no la capa de salida original
# de esta red.
# En base_model tenemos cargado nuestro modelo Xception sin la capa de salida. Nosotros podemos
# poner NUESTRA PROPIA CAPA(S) DE SALIDA
base_model = tf.keras.applications.xception.Xception(weights="imagenet", include_top=False)
# Acá agregamos nuestras capas adicionales
avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(5, activation="softmax")(avg)

# Podemos conectar el input de nuestro modelo base con el output recién generado a través de 
# un modelo de Keras
model = tf.keras.Model(inputs=base_model.input, outputs=output)
# En este punto tenemos un nuevo modelo que utiliza como base la arquitectura Xception

In [ ]:
# También nosotros conversamos que es posible decidir si queremos reentrenar aquellas capas ya entrenadas

# Acá vamos capa por capa modificando el parámetro "trainable" que permite (o no) reentrenar dicha capa
for layer in base_model.layers:
    layer.trainable = False # Esto impide que las capas se re entrenen

# Con este for, sobre todas las capas de nuestro modelo, estamos impidiendo que se reentrene
# alguna de sus capas ya entrenadas

In [9]:
# Proceso de compilación  (tal cual vimos en las clases anteriores)
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", 
              metrics=["accuracy"])

In [11]:
history = model.fit(train_set, epochs=10, validation_data=valid_set)

Epoch 1/10


2023-05-23 20:44:58.759675: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at conv_ops.cc:1120 : NOT_FOUND: No algorithm worked!  Error messages:
  Profiling failure on CUDNN engine 1#TC: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 51838992 bytes.
  Profiling failure on CUDNN engine 1: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 29368704 bytes.
  Profiling failure on CUDNN engine 0#TC: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 16777216 bytes.
  Profiling failure on CUDNN engine 0: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 16777216 bytes.
  Profiling failure on CUDNN engine 2#TC: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 26411008 bytes.
  Profiling failure on CUDNN engine 2: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 26411008 bytes.
  Profiling failure on CUDNN engine 5#TC: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 553779200 bytes.
  Profiling failure on C

NotFoundError: Graph execution error:

Detected at node 'model/block14_sepconv2/separable_conv2d' defined at (most recent call last):
    File "/home/polivares/anaconda3/envs/DataScience/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/polivares/anaconda3/envs/DataScience/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/polivares/anaconda3/envs/DataScience/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/polivares/anaconda3/envs/DataScience/lib/python3.8/site-packages/traitlets/config/application.py", line 982, in launch_instance
      app.start()
    File "/home/polivares/anaconda3/envs/DataScience/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/home/polivares/anaconda3/envs/DataScience/lib/python3.8/site-packages/tornado/platform/asynci

In [ ]:
model.predict(test_set)

In [ ]:
for image, label in test_set:
    print("Etiqueta real",label)
    print("Predicción", model.predict(image))
